# Snek Demo

Snek is developed to improve upon the original cobra package by making it easier to access and less prone for unintended errors.

In [1]:
import cobra 
import snek
model = cobra.io.load_model('textbook')

## 1. Ensure Correct Spelling

When setting bounds of a reaction or the objective function with the main CobraPy package, no error is raised when a spelling error occurs.

In [2]:
with model:
    model.reactions.EX_glc__D_e.lower_bound = -5
    print('{:5.2f}'.format(model.slim_optimize()))
    model.reactions.EX_glc__D_e.lower_buond = -10
    # no error is raised but bound is not updated.
    print('{:5.2f}'.format(model.slim_optimize()))

 0.42
 0.42


This can happen frequently and is hard to debug. Also, the officially recommended way of setting exchange bounds needs 3 lines instead of 1.

In [3]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 5
    model.medium = medium
    print('{:5.2f}'.format(model.slim_optimize()))
    medium["EX_glc__D_e"] = 10
    model.medium = medium
    print('{:5.2f}'.format(model.slim_optimize()))

 0.42
 0.87


The following snek.core functions ensure correct spelling and are still one-liners for efficient coding.

In [4]:
with model:
    model = snek.set_bounds(model,'EX_glc__D_e',-5)
    print('{:5.2f}'.format(model.slim_optimize()))
    model = snek.set_bounds(model,'EX_glc__D_e',-10)
    print('{:5.2f}'.format(model.slim_optimize()))

 0.42
 0.87


The same issue can arise when trying to change the objective function of a model.

In [5]:
with model:
    print('{:5.2f}'.format(model.slim_optimize()))
    model.objctive = 'EX_etoh_e'
    # no error is raised but objective function is not updated.
    print('{:5.2f}'.format(model.slim_optimize()))
    model = snek.set_objective(model,'EX_etoh_e','max')
    print('{:5.2f}'.format(model.slim_optimize()))

 0.87
 0.87
20.00


## 2. Checking for Logical Errors

CobraPy gives the user a lot of freedom. However, here we implemented some basic checks that ensure that no logical errors are made and prints warning if this is true.
This logical errors are sometimes intended, therefore only warnings are printed, however, we think that a more verbose output can help researchers when exploring the model.

In [6]:
with model:
    model.objective = 'ATPM'
    model.objective_direction = 'min'
    print('{:5.2f}'.format(model.slim_optimize()))
    # here we try to minimize a reaction with a custom lower bound
    # although this might be intended, it often isn't.
    model = snek.set_objective(model,'ATPM','min')
    print('{:5.2f}'.format(model.slim_optimize()))

 8.39
 8.39


/home/users/mgotsmy/.conda/envs/2206test3.10/lib/python3.10/site-packages/snek-0.1-py3.10.egg/snek/core.py:43: UserWarning: The objective reaction has a lower bound at 8.39 and a upper bound at 1000.00. This could lead to problems during optimization.
  warnings.warn('The objective reaction has a lower bound at {:.2f} and a upper bound at {:.2f}. This could lead to problems during optimization.'.format(model.reactions.get_by_id(reaction).lower_bound,model.reactions.get_by_id(reaction).upper_bound))


In [9]:
with model:
    model.objective = 'ATPM'
    model.objective_direction = 'min'
    print('{:5.2f}'.format(model.slim_optimize()))
    # here we try to minimize a reaction with a custom lower bound
    # although this might be intended, it often isn't.
    model = snek.set_objective(model,'ATPM','min')
    print('{:5.2f}'.format(model.slim_optimize()))

 8.39
 8.39
